# A Motif-based Perspective on Network Structue and it's Influence on Epidemic Behaviour

In [1140]:
import numpy as np
import scipy.linalg as lg
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import EoN
from EoN import Simulation_Investigation
import scipy.special
import pandas as pd
import operator as op
from scipy import stats
from scipy.stats import linregress
from scipy.stats import expon
from itertools import islice
from scipy.integrate import odeint
from matplotlib.lines import Line2D
from scipy.stats import shapiro
%config InlineBackend.figure_format = 'svg'

In [7]:
n = 1000
net = 180
net_per_group = 30
group = 6
no_tot_sim = 5400
gamma = 0.076
tau = 0.06
tmax = 365
n = 1000
no_sim = 30
sim_p_group = 900
ex_ep = 10

Note we refer to network groups, this we use to differentiate between networks that generated with the different $\xi$ values and the control group of Barabasi Albert networks

# Calulating the number of triangle-based motifs

Create a list with every filename representing the name of each contact network we will use. BAmatrix represents Barabasi Albert networks, BAplusTN represents our modified networks. We include the number of the network and the tr value used to generate the network in MATLAB where tr = triangles 

In [65]:
filenames = []
for i in range(1,31):
    BAmatrix = 'BAmatrix'+'_'+ str(i) +'.txt'
    filenames.append(BAmatrix)
for j in range(31,61):
    BAplusTN = 'BAplusTN'+'_'+ str(j)+'_'+ str(500)+ '.txt'
    filenames.append(BAplusTN)
for k in range(61,91):
    BAplusTN = 'BAplusTN'+'_'+ str(k)+'_'+str(750)+ '.txt'
    filenames.append(BAplusTN)
for l in range(91,121):
    BAplusTN = 'BAplusTN'+'_'+ str(l)+'_'+str(1000)+'.txt'
    filenames.append(BAplusTN)
for p in range(121,151):
    BAplusTN = 'BAplusTN'+'_'+ str(p)+'_'+ str(1250)+'.txt'
    filenames.append(BAplusTN)
for t in range(151,181):
    BAplusTN = 'BAplusTN'+'_'+ str(t)+'_'+ str(1500)+ '.txt'
    filenames.append(BAplusTN)

Read csv files for each network, calculate the number of triangle-based motifs in each. Store in 4 lists for each motif

In [4]:
Triangle_motif = []
Diamond_motif = []
Tadpole_motif = []
Bowtie_motif = []
for i in range(net):
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        
        
        TRI_motif = (1/6)*np.trace(adj_i**3)
        Triangle_motif.append(TRI_motif)
        
        adj_i_2 = adj_i**2 
        DI_motif = (1/4)*np.sum([(adj_i_2[k,j]*adj_i[k,j])*((adj_i_2[k,j]*adj_i[k,j])-1) for k in range(n) for j in range(n)])
        Diamond_motif.append(DI_motif)

        

        adj_i_3 = adj_i**3  
        s = adj_i.sum(axis = 1)       
        TAD_motif = np.sum([(1/2)*(adj_i_3[k,k])*(int(s[k]) - 2) for k in range(n) if int(s[k]) >2])
        Tadpole_motif.append(TAD_motif)
        
    
        BOW_motif = (1/2)*(np.sum([(1/2)*(adj_i_3[k,k])*((1/2)*(adj_i_3[k,k])-(1)) for k in range(n)])) - 2*Diamond_motif[i]
        Bowtie_motif.append(BOW_motif)

Store number of triangle based motifs per network in table form

In [991]:
data = {'Triangle motif':  Triangle_motif ,
        'Diamond motif': Diamond_motif,
        'Tadpole motif': Tadpole_motif,
        'Bowtie motif': Bowtie_motif}
df = pd.DataFrame(data, index = filenames)
df.to_csv('Number_of_motifs_per_network.csv', index=True) 

In [4]:
motifs = pd.read_csv('Number_of_motifs_per_network.csv')
Triangle_motif = motifs["Triangle motif"]
Diamond_motif = motifs["Diamond motif"]
Tadpole_motif = motifs["Tadpole motif"]
Bowtie_motif = motifs["Bowtie motif"]

Now we will create a lists for each group with the average number of each motif

In [8]:
tri = np.array_split(np.array(Triangle_motif), group)
av_tri_1 = round(np.sum(tri[0])/net_per_group)
av_tri_2 = round(np.sum(tri[1])/net_per_group)
av_tri_3 = round(np.sum(tri[2])/net_per_group)
av_tri_4 = round(np.sum(tri[3])/net_per_group)
av_tri_5 = round(np.sum(tri[4])/net_per_group)
av_tri_6 = round(np.sum(tri[5])/net_per_group)
av_tri = [av_tri_1,av_tri_2,av_tri_3,av_tri_4,av_tri_5,av_tri_6]

di = np.array_split(np.array(Diamond_motif), group)
av_di_1 = round(np.sum(di[0])/net_per_group)
av_di_2 = round(np.sum(di[1])/net_per_group)
av_di_3 = round(np.sum(di[2])/net_per_group)
av_di_4 = round(np.sum(di[3])/net_per_group)
av_di_5 = round(np.sum(di[4])/net_per_group)
av_di_6 = round(np.sum(di[5])/net_per_group)
av_di = [av_di_1,av_di_2,av_di_3,av_di_4,av_di_5,av_di_6]

tad = np.array_split(np.array(Tadpole_motif), group)
av_tad_1 = round(np.sum(tad[0])/net_per_group)
av_tad_2 = round(np.sum(tad[1])/net_per_group)
av_tad_3 = round(np.sum(tad[2])/net_per_group)
av_tad_4 = round(np.sum(tad[3])/net_per_group)
av_tad_5 = round(np.sum(tad[4])/net_per_group)
av_tad_6 = round(np.sum(tad[5])/net_per_group)
av_tad = [av_tad_1,av_tad_2,av_tad_3,av_tad_4,av_tad_5,av_tad_6]


bow = np.array_split(np.array(Bowtie_motif), group)
av_bow_1 = round(np.sum(bow[0])/net_per_group)
av_bow_2 = round(np.sum(bow[1])/net_per_group)
av_bow_3 = round(np.sum(bow[2])/net_per_group)
av_bow_4 = round(np.sum(bow[3])/net_per_group)
av_bow_5 = round(np.sum(bow[4])/net_per_group)
av_bow_6 = round(np.sum(bow[5])/net_per_group)
av_bow = [av_bow_1,av_bow_2,av_bow_3,av_bow_4,av_bow_5,av_bow_6]

Store average number of motifs per group into a table

In [992]:
data = {'Average # Triangles':  av_tri ,
        'Average # Diamonds': av_di,
        'Average # Tadpoles': av_tad,
        'Average # Bowties': av_bow}
index_titles = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
df2 = pd.DataFrame(data, index = index_titles )
df2.to_csv('Average_#_of_motifs_per_network.csv', index=True) 

### Plot comparissons of motifs

Triangle vs Diamond, Triangle vs Tadpole, Triangle vs Bowtie

In [2074]:
plt.scatter(Triangle_motif, Diamond_motif, color = "black")
plt.title('Triangle motifs vs Diamond motifs')
plt.ylabel('Number of diamond motifs')
plt.xlabel('The number of triangle motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Triangle_vs_Diamond",dpi=300, bbox_inches='tight')
plt.close()


plt.scatter(Triangle_motif, Tadpole_motif, color = "black")
plt.title('Triangle motifs vs Tadpole motifs')
plt.ylabel('Number of tadpole motifs')
plt.xlabel('Number of triangle motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Triangle_vs_Tadpole",dpi=300, bbox_inches='tight')
plt.close()


plt.scatter(Triangle_motif, Bowtie_motif, color = "black")
plt.title('Triangle motifs vs Bowtie motifs')
plt.ylabel('Number of bowtie motifs')
plt.xlabel('Number of triangle motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Triangle_vs_Bowtie",dpi=300, bbox_inches='tight')
plt.close()

Diamond vs Bowtie, Diamond vs Tadpole

In [2073]:
plt.scatter(Diamond_motif, Bowtie_motif,color = "black")
plt.title('Diamond motifs vs Bowtie motifs')
plt.ylabel('Number of bowtie motifs')
plt.xlabel('Number of diamond motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Diamond_vs_Bowtie",dpi=300, bbox_inches='tight')
plt.close()
plt.scatter(Diamond_motif, Tadpole_motif,color = "black")
plt.title('Diamond motifs vs Tadpole motifs')
plt.ylabel('Number of tadpole motifs')
plt.xlabel('Number of diamond motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Diamond_vs_Tadpole",dpi=300, bbox_inches='tight')
plt.close()

Bowtie vs Tadpole

In [2072]:
plt.scatter(Bowtie_motif,Tadpole_motif,color = "black")
plt.title('Bowtie motifs vs Tadpole Motifs')
plt.ylabel('Number of  tadpole motifs')
plt.xlabel('Number of  bowtie motifs')
plt.xlim(xmin = 0)
plt.ylim(ymin = 0)
plt.savefig("Tadpole_vs_Bowtie",dpi=300, bbox_inches='tight')
plt.close()

# Simulation of SIR epidemics on networks

Create a list for of filenames for simulation results. 30 simulations per network. 

In [11]:
filenames2 = []
for i in range(net):
    for j in range(net_per_group):
        name = 'Simulation_parameters'+str(i)+'_'+str(j)+'.csv'
        filenames2.append(name)

In [13]:
for i in range(net):
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        
        for j in range(no_sim):
            G = nx.from_numpy_matrix(adj_i)
            t, S, I, R = EoN.fast_SIR(G, tau, gamma,
                            initial_infecteds = None, rho = None)
            
            data = {'Time':  t ,
                    'Susceptible': S,
                    'Infected': I,
                    'Recovered': R}
            df = pd.DataFrame(data)
            df["Days"] = np.floor(df["Time"])
            df.to_csv('Simulation_parameters'+str(i)+'_'+str(j)+'.csv', index=True) 

## Epidemic Size

 Calculate the epidemic size for each simulation:
 
 Epidemic size = $S_{Initial} - S_{Final} +1$

In [2088]:
Epidemic_size = []
for i in range(no_tot_sim):
        data = pd.read_csv(filenames2[i])
        S_initial = list(data["Susceptible"])[0]
        S_final = list(data["Susceptible"])[-1]
        Epidemic = (S_initial - S_final)+1
        Epidemic_size.append(Epidemic)

The epidemic size for simulations per network 

In [2091]:
Epidemics = np.array_split(np.array(Epidemic_size), net)

Separate results for epidemics that take off and that don't.

In [15]:
Epidemics_more_than_10 = []
for i in range(net):
    Epidemics_pnn=[]
    for k in Epidemics[i]:
        if k > ex_ep:
            Epidemics_pnn.append(k)
    Epidemics_more_than_10.append(Epidemics_pnn)

Average epidemics sizes > 10 per network

In [16]:
av_ep_size_ex_10_pn = []
for i in range(net):
    av_ep_ex_10 = np.sum(Epidemics_more_than_10[i])/len(Epidemics_more_than_10[i])
    av_ep_size_ex_10_pn.append(av_ep_ex_10)

Plots for the average epidemic size per network and number of triangle-based motifs for each network, excluding 10

In [2131]:
#plt.scatter(Triangle_motif,av_ep_size_pn)
plt.scatter(Triangle_motif,av_ep_size_ex_10_pn,color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Average Epidemic Size")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
plt.savefig("Triangle_motifs_vs_Average_ep_size_ex_10",dpi=300, bbox_inches='tight')
#plt.title("Number of Triangle motifs vs Average Epidemic Size per network ($>10$)")
plt.close()

plt.scatter(Diamond_motif,av_ep_size_ex_10_pn,color = "black")
plt.xlabel("Number of diamond motifs")
plt.ylabel("Average Epidemic Size")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
plt.savefig("Diamond_motifs_vs_Average_ep_size_ex_10")
#plt.title("Number of Diamond motifs vs Average Epidemic Size per network ($>10$)")
plt.close()

plt.scatter(Tadpole_motif,av_ep_size_ex_10_pn,color = "black")
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Average Epidemic Size")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
plt.savefig("Tadpole_motifs_vs_Average_ep_size_ex_10")
plt.title("Number of Tadpole motifs vs Average Epidemic Size per network ($>10$)")
plt.close()

plt.scatter(Bowtie_motif,av_ep_size_ex_10_pn,color = "black")
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Average Epidemic Size")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
plt.savefig("Bowtie_motifs_vs_Average_ep_size_ex_10")
plt.title("Number of Bowtie motifs vs Average Epidemic Size per network ($>10$)")
plt.close()

Store the data in a table

In [2092]:
Network = list(range(1,net+1))
data = {'Average Epidemic size per network(excl. 10)': av_ep_size_ex_10_pn,
        'Triangle motif':  Triangle_motif ,
        'Diamond motif': Diamond_motif,
        'Tadpole motif': Tadpole_motif,
        'Bowtie motif': Bowtie_motif}
df = pd.DataFrame(data)
df.to_csv('Average_epidemic_size_pn_vs_motifs.csv', index = Network) 

Create tables for average epidemic size for control networks and networks with the same $\xi$ value

Average epidemic size excluding sizes below and equal to 10

In [28]:
Av_Epidemic_size_ex_10_pg = list(np.array_split(np.array(av_ep_size_ex_10_pn), group)) 
Av_Epidemic_size_ex_10_G1 =list(Av_Epidemic_size_ex_10_pg[0])
Av_Epidemic_size_ex_10_G2 =list(Av_Epidemic_size_ex_10_pg[1])
Av_Epidemic_size_ex_10_G3 =list(Av_Epidemic_size_ex_10_pg[2])
Av_Epidemic_size_ex_10_G4 =list(Av_Epidemic_size_ex_10_pg[3])
Av_Epidemic_size_ex_10_G5 =list(Av_Epidemic_size_ex_10_pg[4])
Av_Epidemic_size_ex_10_G6 =list(Av_Epidemic_size_ex_10_pg[5])

In [29]:
Triangle_motif_pg = list(np.array_split(np.array(Triangle_motif), group)) 
Triangle_motif_G1 = list(Triangle_motif_pg[0])
Triangle_motif_G2 = list(Triangle_motif_pg[1])
Triangle_motif_G3 = list(Triangle_motif_pg[2])
Triangle_motif_G4 = list(Triangle_motif_pg[3])
Triangle_motif_G5 = list(Triangle_motif_pg[4])
Triangle_motif_G6 = list(Triangle_motif_pg[5])

In [30]:
Diamond_motif_pg = list(np.array_split(np.array(Diamond_motif), group))
Diamond_motif_G1 = list(Diamond_motif_pg[0])
Diamond_motif_G2 = list(Diamond_motif_pg[1])
Diamond_motif_G3 = list(Diamond_motif_pg[2])
Diamond_motif_G4 = list(Diamond_motif_pg[3])
Diamond_motif_G5 = list(Diamond_motif_pg[4])
Diamond_motif_G6 = list(Diamond_motif_pg[5])

In [31]:
Tadpole_motif_pg = list(np.array_split(np.array(Tadpole_motif), group)) 
Tadpole_motif_G1 = list(Tadpole_motif_pg[0])
Tadpole_motif_G2 = list(Tadpole_motif_pg[1])
Tadpole_motif_G3 = list(Tadpole_motif_pg[2])
Tadpole_motif_G4 = list(Tadpole_motif_pg[3])
Tadpole_motif_G5 = list(Tadpole_motif_pg[4])
Tadpole_motif_G6 = list(Tadpole_motif_pg[5])

In [32]:
Bowtie_motif_pg = list(np.array_split(np.array(Bowtie_motif), group)) 
Bowtie_motif_G1 = list(Bowtie_motif_pg[0])
Bowtie_motif_G2 = list(Bowtie_motif_pg[1])
Bowtie_motif_G3 = list(Bowtie_motif_pg[2])
Bowtie_motif_G4 = list(Bowtie_motif_pg[3])
Bowtie_motif_G5 = list(Bowtie_motif_pg[4])
Bowtie_motif_G6 = list(Bowtie_motif_pg[5])

In [33]:
ind = list(range(1,net_per_group+1))
data1 = {'Average Epidemic Size ':  Av_Epidemic_size_G1  ,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G1,
        'Triangle motif':  Triangle_motif_G1,
        'Diamond motif': Diamond_motif_G1,
        'Tadpole motif': Tadpole_motif_G1,
        'Bowtie motif': Bowtie_motif_G1}
df1 = pd.DataFrame(data1)
df1.to_csv('Average_epidemic_size_G1_vs_motifs.csv', index = ind) 

In [34]:
ind = list(range(1,net_per_group+1))
data2 = {'Average Epidemic Size ':  Av_Epidemic_size_G2  ,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G2,
        'Triangle motif':  Triangle_motif_G2,
        'Diamond motif': Diamond_motif_G2,
        'Tadpole motif': Tadpole_motif_G2,
        'Bowtie motif': Bowtie_motif_G2}
df2 = pd.DataFrame(data2)
df2.to_csv('Average_epidemic_size_G2_vs_motifs.csv', index = ind) 

In [35]:
ind = list(range(1,net_per_group+1))
data3 = {'Average Epidemic Size ':  Av_Epidemic_size_G3 ,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G3,
        'Triangle motif':  Triangle_motif_G3,
        'Diamond motif': Diamond_motif_G3,
        'Tadpole motif': Tadpole_motif_G3,
        'Bowtie motif': Bowtie_motif_G3}
df3 = pd.DataFrame(data3)
df3.to_csv('Average_epidemic_size_G3_vs_motifs.csv', index = ind) 

In [36]:
ind = list(range(1,net_per_group+1))
data4 = {'Average Epidemic Size ':  Av_Epidemic_size_G4,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G4,
        'Triangle motif':  Triangle_motif_G4,
        'Diamond motif': Diamond_motif_G4,
        'Tadpole motif': Tadpole_motif_G4,
        'Bowtie motif': Bowtie_motif_G4}
df4 = pd.DataFrame(data4)
df4.to_csv('Average_epidemic_size_G4_vs_motifs.csv', index = ind) 

In [37]:
ind = list(range(1,net_per_group+1))
data5 = {'Average Epidemic Size ':  Av_Epidemic_size_G5,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G5,
        'Triangle motif':  Triangle_motif_G5,
        'Diamond motif': Diamond_motif_G5,
        'Tadpole motif': Tadpole_motif_G5,
        'Bowtie motif': Bowtie_motif_G5}
df5 = pd.DataFrame(data5)
df5.to_csv('Average_epidemic_size_G5_vs_motifs.csv', index = ind) 

In [38]:
ind = list(range(1,net_per_group+1))
data6 = {'Average Epidemic Size ':  Av_Epidemic_size_G6,
         'Average Epidemic size per network(excl. 10)': Av_Epidemic_size_ex_10_G6,
        'Triangle motif':  Triangle_motif_G6,
        'Diamond motif': Diamond_motif_G6,
        'Tadpole motif': Tadpole_motif_G6,
        'Bowtie motif': Bowtie_motif_G6}
df6 = pd.DataFrame(data6)
df6.to_csv('Average_epidemic_size_G6_vs_motifs.csv', index = ind)

Calculate variance, std and percentiles for epidemic size per network

For all epidemics 

In [22]:
Epidemic_size_pn =list(np.array_split(np.array(Epidemic_size), net)) 
epsize_variance = []
percentile_95 = []
percentile_5 =[]
std = []
for i in range(net):
    var = np.var(Epidemic_size_pn[i])
    epsize_variance.append(var)
    per95 = np.percentile(Epidemic_size_pn[i],95)
    percentile_95.append(per95)
    per5 = np.percentile(Epidemic_size_pn[i],5)
    percentile_5.append(per5)
    sd = np.std(Epidemic_size_pn[i], ddof=1)/np.sqrt(len(Epidemic_size_pn[i]))
    std.append(sd)

For epidemics that took hold

In [23]:
epsize_variance_ex_10= []
percentile_95_ex_10 = []
percentile_5_ex_10 =[]
std_ex_10 = []
for i in range(net):
    var1 = np.var(Epidemics_more_than_10[i])
    epsize_variance_ex_10.append(var1)
    per951 = np.percentile(Epidemics_more_than_10[i] , 95)
    percentile_95_ex_10.append(per951)
    per51 = np.percentile(Epidemics_more_than_10[i], 5)
    percentile_5_ex_10.append(per51)
    sd1 = np.std(Epidemics_more_than_10[i])/np.sqrt(len(Epidemics_more_than_10[i]))
    std_ex_10.append(sd1)

In [1522]:
max_range_ep_size = [(min(x),max(x)) for x in Epidemics_more_than_10]

In [1534]:
max_range_ep_size_pg = list(np.array_split(np.array(max_range_ep_size), group))
#max_range_ep_size_pg

Save data

In [24]:
ind = list(range(1,net+1))
data8 = {'Average Epidemic Size per network': av_ep_size_pn,
        'Variance':  epsize_variance,
        '5th percentile': percentile_5,
        '95th percentile': percentile_95,
        'Standard deviation': std}
df8 = pd.DataFrame(data8)
df8.to_csv('Average_Epidemic_size_vs_variance.csv', index = ind)

In [25]:
ind = list(range(1,net+1))
data8 = {'Average Epidemic Size per network excl. 1 or 2': av_ep_size_ex_10_pn,
        'Variance':  epsize_variance_ex_10,
        '5th percentile': percentile_5_ex_10,
        '95th percentile': percentile_95_ex_10,
        'Standard deviation': std_ex_10}
df8 = pd.DataFrame(data8)
df8.to_csv('Average_Epidemic_size_ex_1_2_vs_variance.csv', index = ind)

Plot average epidemic size > 10 for for networks vs motifs, including percentiles 

In [26]:
yerr01 = [(np.array(av_ep_size_ex_10_pn) - np.array(percentile_5_ex_10)),(np.array(percentile_95_ex_10) - np.array(av_ep_size_ex_10_pn))]


In [2132]:
plt.errorbar(Triangle_motif, av_ep_size_ex_10_pn, yerr = yerr01 , markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.ylabel("Average Epidemic Size")
plt.xlabel("Number of triangle motifs")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.savefig("Triangle_motifs_vs_Average_ep_size_ex_10_PERCENTILES", dpi = 300, bbox_inches='tight')
plt.title("Average epidemic size per network ($>10$)")
plt.close()

plt.errorbar(Diamond_motif, av_ep_size_ex_10_pn, yerr = yerr01 , markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
#plt.plot(Diamond_motif, slope*(Diamond_motif)+intercept, '--k'  )
plt.ylabel("Average Epidemic Size")
plt.xlabel("Number of diamond motifs")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.savefig("Diamond_motifs_vs_Average_ep_size_ex_10_PERCENTILES",dpi=300, bbox_inches='tight')
plt.title("The number of diamond motifs vs average epidemic size per network ($>10$)")
plt.close()

plt.errorbar(Tadpole_motif, av_ep_size_ex_10_pn, yerr = yerr01 , markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.ylabel("Average Epidemic Size")
plt.xlabel("Number of tadpole motifs")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.savefig("Tadpole_motifs_vs_Average_ep_size_ex_10_PERCENTILES",dpi=300, bbox_inches='tight')
plt.title("Average epidemic Size per network ($>10$)")
plt.close()

plt.errorbar(Bowtie_motif, av_ep_size_ex_10_pn, yerr = yerr01 , markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.ylabel("Average Epidemic Size")
plt.xlabel("Number of bowtie motifs")
plt.ylim(ymin = 0, ymax = 1000)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.savefig("Bowtie_motifs_vs_Average_ep_size_ex_10_PERCENTILES",dpi=300, bbox_inches='tight')
plt.title("Average epidemic size per network ($>10$)")
plt.close()


Testing correlation coefficients and significance for the average epidemic size per network and the number of trinagle-based motifs per network

In [994]:
slope1, intercept1, r1, p1, se = linregress(Triangle_motif, av_ep_size_ex_10_pn)
slope2, intercept3, r2, p2, se = linregress(Diamond_motif, av_ep_size_ex_10_pn)
slope3, intercept3, r3, p3, se = linregress(Tadpole_motif, av_ep_size_ex_10_pn)
slope4, intercept4, r4, p4, se = linregress(Bowtie_motif, av_ep_size_ex_10_pn)

In [1549]:
#r1,r2,r3,r4,p1,p2,p3,p4

## Average peak incidence per network

Create files for daily incidence and rolling means for each simulation. Found the number of infections per time step and grouped by day to sum infections per day

In [1628]:
for j in range(no_tot_sim):
    daily_incidence = []
    mean_rolling_incidence = []
    sample = pd.read_csv(filenames2[j])
    Days = list(sample["Days"])
    Recover_pts = list(sample["Recovered"].diff(periods = 1))
    incidence = (-1)*np.array(sample["Susceptible"].diff(periods = 1))
    
    data = {"Incidence": incidence,"Days": list(sample["Days"])}
    df = pd.DataFrame(data)
    
    d_i = df.groupby("Days", as_index=False)["Incidence"].sum()
    
    d_i["Days"] = d_i["Days"].astype(np.int64)

    d_i = d_i.set_index("Days")
    d_i = d_i.reindex(range(tmax), fill_value = 0)
    d_i = d_i.reset_index()

    d_i["Rolling mean 3"] = d_i["Incidence"].rolling(3, center=True, closed = 'both').mean()
    d_i["Rolling mean 7"] = d_i["Incidence"].rolling(7, center=True, closed = 'both').mean()
    
    d_i.to_csv('Incidence'+str(j)+'.csv', index = list(range(tmax)))

In [1629]:
filenames3 = []
for i in range(no_tot_sim):
    name = 'Incidence'+str(i)+'.csv'
    filenames3.append(name)

Excluded epidemics that did not take hold

In [1630]:
filenames_no_epidemic = []
index_no_ep = []
for y in [i for i, x in enumerate(Epidemic_size) if x <= ex_ep]:
    t = 'Incidence'+str(y)+'.csv'
    index_no_ep.append(y)
    filenames_no_epidemic.append(t)

In [1631]:
len(filenames_no_epidemic)

1798

Peak_incidence and time for peak incidence for epidemics that took hold

In [1632]:
filenames3_1 = [x for x in filenames3 if x not in filenames_no_epidemic]

In [1735]:
range(len(filenames3_1))

range(0, 3602)

In [1737]:
peak_incidence = []
time_peak_incidence_1= []

for k in range(len(filenames3_1)):
    mysample = pd.read_csv(filenames3_1[k])
    t = mysample["Rolling mean 7"]
    max_t = np.max(t)
    time_p_incidence =  mysample[mysample["Rolling mean 7"] == max_t].index.values
    
    peak_incidence.append(max_t)
    time_peak_incidence_1.append(time_p_incidence)

In [1701]:
time_peak_incidence = []
for i in time_peak_incidence_1:
    if len(i)>1:
        time_peak_incidence.append(i[0])
    else:
        time_peak_incidence.append(int(i))

In [1950]:
peak_incidence_rolling_mean_3 = []
time_peak_incidence_1_rolling_mean_3= []

for k in range(len(filenames3_1)):
    mysample = pd.read_csv(filenames3_1[k])
    t = mysample["Rolling mean 3"]
    max_t = np.max(t)
    time_p_incidence =  mysample[mysample["Rolling mean 3"] == max_t].index.values
    
    peak_incidence_rolling_mean_3.append(max_t)
    time_peak_incidence_1_rolling_mean_3.append(time_p_incidence)

In [1951]:
time_peak_incidence_rolling_mean_3 = []
for i in time_peak_incidence_1_rolling_mean_3:
    if len(i)>1:
        time_peak_incidence_rolling_mean_3.append(i[0])
    else:
        time_peak_incidence_rolling_mean_3.append(int(i))

In [1925]:
shapiro(peak_incidence)[1] > 0.05

False

In [1926]:
min(peak_incidence), max(peak_incidence)

(1.0, 64.75)

Plots for daily incidence per epidemics that took hold vs time for networks with the same $\xi$ value and control networks

In [1662]:
y = [len(x) for x in Epidemics_more_than_10] #number of epidemics that took hold per network
y_pg = list(np.array_split(np.array(y), group)) # split the number of epidemics that took hold pn to pg
yy_pg = [np.sum(x) for x in y_pg] # total number of epidemics that took hold per group
filenames3_1_pg =  [list(islice(iter(filenames3_1), elem)) for elem in yy_pg] # split filenames into filenames pg

In [1636]:
myrange = np.cumsum(yy_pg) #range for epidemics that took hold per group
myrange

array([ 624, 1258, 1908, 2496, 3083, 3602])

In [2032]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []
index_max = []

df = []



for j in range(myrange[0]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df.append(df1)
    
    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)
    

combinded = pd.concat(df,axis=1)
combinded.to_csv('Incidence_vs_time_bara.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.ylabel("Daily Incidence")
plt.xlabel("Time")
plt.savefig("Graph_Incidence_vs_time_barab",dpi=300, bbox_inches='tight' )
plt.close()

In [2057]:
df11 = combinded.median(axis = 1)
df11_2_5 = combinded.quantile(q= 0.025, axis=1)
df11_25 = combinded.quantile(q= 0.25, axis=1)
df11_75 = combinded.quantile(q= 0.75, axis=1)
df11_97_5 = combinded.quantile(q= 0.975, axis=1)

plt.plot(range(len(df11)),list(df11), label= "Median")
plt.fill_between(range(len(df11)), np.array(df11)- (np.array(df11) - np.array(df11_25)), np.array(df11)+(np.array(df11_75) - np.array(df11)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF', label =  r'$50\%$'" interval")
plt.fill_between(range(len(df11)), np.array(df11)- (np.array(df11) - np.array(df11_2_5)), np.array(df11)+(np.array(df11_97_5) - np.array(df11)), alpha =0.2, label = r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin = 0, xmax = 100)
plt.ylim(ymin = 0, ymax = 80)
plt.title("Control")
plt.savefig("Daily incidence vs time bara",dpi=300, bbox_inches='tight')
plt.close()

In [2037]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []
index_max = []

df2 = []
for j in range(myrange[0],myrange[1]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df2.append(df1)

    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)
    
combinded2 = pd.concat(df2,axis=1)
combinded2.to_csv('Incidence_vs_time_250.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.ylabel("Daily Incidence")
plt.xlabel("Time")
plt.savefig("Graph_Incidence_vs_time_500",dpi=300, bbox_inches='tight' )
plt.close()

In [2056]:
df22 = combinded2.median(axis = 1)
df22_2_5 = combinded2.quantile(q= 0.025, axis=1)
df22_25 = combinded2.quantile(q= 0.25, axis=1)
df22_75 = combinded2.quantile(q= 0.75, axis=1)
df22_97_5 = combinded2.quantile(q= 0.975, axis=1)

plt.plot(range(len(df22)),list(df22), label = 'Median')
plt.fill_between(range(len(df22)), np.array(df22)- (np.array(df22) - np.array(df22_25)), np.array(df22)+(np.array(df22_75) - np.array(df22)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF', label = r'$50\%$'" interval")
plt.fill_between(range(len(df22)), np.array(df22)- (np.array(df22) - np.array(df22_2_5)), np.array(df22)+(np.array(df22_97_5) - np.array(df22)), alpha =0.2, label = r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin = 0, xmax = 100)
plt.ylim(ymin = 0, ymax = 80)
plt.title(r'$\xi = 500$')
plt.savefig("Daily incidence vs time five",dpi=300, bbox_inches='tight')
plt.close()

In [2039]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []
index_max = []

df3 = []
# array([ 624, 1258, 1908, 2496, 3083, 3602])
  
    
for j in range(myrange[1],myrange[2]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df3.append(df1)

    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)
    
#combinded3 = pd.concat(df3,axis=1)
#combinded3.to_csv('Incidence_vs_time_750.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.ylabel("Daily Incidence")
plt.xlabel("Time")
plt.savefig("Graph_Incidence_vs_time_750",dpi=300, bbox_inches='tight' )
plt.close()

In [2027]:
myrange

array([ 624, 1258, 1908, 2496, 3083, 3602])

In [2070]:
plot_start_index = 0
plot_end_index = len(filenames3_1)
n_plots = int(round(len(filenames3_1)/10,0))

for plot_num in range(n_plots):
    fig = plt.figure()
    plt.ion()
    ax = fig.add_subplot(111)
    currentNsmax_max = []
    index_max = []

    df3 = []
    start_j = plot_start_index + plot_num * 10
    end_j = plot_start_index + (plot_num+1) * 10
    
    for j in range(start_j, end_j):
        sample = pd.read_csv(filenames3_1[j])
        df1 = sample["Incidence"]
        df3.append(df1)

        currentNsmax = max(df1)
        if(currentNsmax < 51):
            lines = Line2D(list(range(len(df1))), df1, color = "grey")
        else:
            lines = Line2D(list(range(len(df1))), df1, color = "grey")
        
        ax.add_line(lines)
        plt.ioff()
        currentNsmax_max.append(currentNsmax)
        
    ax.set_xlim(0.0, 100)
    ax.set_ylim(0.0, 100)
    plt.ylabel("Daily Incidence")
    plt.xlabel("Time")
    plt.savefig("G6_at_a_time_plots_"+str(j), dpi=300, bbox_inches='tight')
    plt.close()

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [2055]:
df33 = combinded3.median(axis = 1)
df33_2_5 = combinded3.quantile(q= 0.025, axis=1)
df33_25 = combinded3.quantile(q= 0.25, axis=1)
df33_75 = combinded3.quantile(q= 0.75, axis=1)
df33_97_5 = combinded3.quantile(q= 0.975, axis=1)

plt.plot(range(len(df33)),list(df33), label = 'Median')
plt.fill_between(range(len(df33)), np.array(df33)- (np.array(df33) - np.array(df33_25)), np.array(df33)+(np.array(df33_75) - np.array(df33)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF',  label = r'$50\%$'" interval")
plt.fill_between(range(len(df33)), np.array(df33)- (np.array(df33) - np.array(df33_2_5)), np.array(df33)+(np.array(df33_97_5) - np.array(df33)), alpha =0.2, label = r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.xlim(xmin = 0, xmax = 100)
plt.ylim(ymin = 0, ymax = 80)
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.title(r'$\xi = 750$')
plt.savefig("Daily incidence vs time seven",dpi=300, bbox_inches='tight')
plt.close()

In [2044]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []
index_max = []

df4 = []
for j in range(myrange[2],myrange[3]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df4.append(df1)
    
    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)

combinded4 = pd.concat(df4,axis=1)
combinded4.to_csv('Incidence_vs_time_1000.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.xlabel('Time')
plt.ylabel('Daily Incidence')
plt.savefig("Graph_Incidence_vs_time_1000",dpi=300, bbox_inches='tight' )
plt.close()

In [2054]:
df44 = combinded4.median(axis = 1)
df44_2_5 = combinded4.quantile(q= 0.025, axis=1)
df44_25 = combinded4.quantile(q= 0.25, axis=1)
df44_75 = combinded4.quantile(q= 0.75, axis=1)
df44_97_5 = combinded4.quantile(q= 0.975, axis=1)


plt.plot(range(len(df44)),list(df44), label = 'Median')
plt.fill_between(range(len(df44)), np.array(df44)- (np.array(df44) - np.array(df44_25)), np.array(df44)+(np.array(df44_75) - np.array(df44)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF', label =  r'$50\%$'" interval")
plt.fill_between(range(len(df44)), np.array(df44)- (np.array(df44) - np.array(df44_2_5)), np.array(df44)+(np.array(df44_97_5) - np.array(df44)), alpha =0.2, label =  r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.title(r'$\xi = 1000$')
plt.xlim(xmin = 0, xmax = 100)
plt.ylim(ymin = 0, ymax = 80)
plt.savefig("Daily incidence vs time one thou not at 365", dpi=300, bbox_inches='tight')
plt.close()

In [2047]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []


df5 = []
for j in range(myrange[3],myrange[4]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df5.append(df1)
    
    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)


combinded5 = pd.concat(df5,axis=1)
combinded5.to_csv('Incidence_vs_time_1250.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.xlabel('Time')
plt.ylabel('Daily Incidence')
plt.savefig("Graph_Incidence_vs_time_1250",dpi=300, bbox_inches='tight' )
plt.close()

In [2053]:
df55 = combinded5.median(axis = 1)
df55_mean = combinded5.mean(axis = 1)
df55_2_5 = combinded5.quantile(q= 0.025, axis=1)
df55_25 = combinded5.quantile(q= 0.25, axis=1)
df55_75 = combinded5.quantile(q= 0.75, axis=1)
df55_97_5 = combinded5.quantile(q= 0.975, axis=1)


plt.plot(range(len(df55)),list(df55), label = 'Median')
plt.fill_between(range(len(df55)), np.array(df55)- (np.array(df55) - np.array(df55_25)), np.array(df55)+(np.array(df55_75) - np.array(df55)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF', label =  r'$50\%$'" interval")
plt.fill_between(range(len(df55)), np.array(df55)- (np.array(df55) - np.array(df55_2_5)), np.array(df55)+(np.array(df55_97_5) - np.array(df55)), alpha =0.2, label =  r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.ylabel("Daily Incidence")
plt.xlabel("Time")
plt.legend()
plt.xlim(xmin = 0, xmax = 100)
plt.ylim(ymin = 0, ymax = 80)
plt.title(r'$\xi = 1250$')
plt.savefig("Daily incidence vs time one two five",dpi=300, bbox_inches='tight')
plt.close()

In [2049]:
fig = plt.figure()
ax = fig.add_subplot(111)
currentNsmax_max = []
index_max = []

df6 = []
for j in range(myrange[4],myrange[5]):
    sample = pd.read_csv(filenames3_1[j])
    df1 = sample["Incidence"]
    df6.append(df1)
    
    currentNsmax = max(df1)
    lines = Line2D(list(range(len(df1))), df1)
    ax.add_line(lines)
    
    currentNsmax_max.append(currentNsmax)
    index_max.append(len(df1)-1)

#combinded6 = pd.concat(df6,axis=1)
#combinded6.to_csv('Incidence_vs_time_1500.csv', index = list(range(tmax)))

ax.set_xlim(0.0, 200)
ax.set_ylim(0.0, max(currentNsmax_max))
plt.xlabel('Time')
plt.ylabel('Daily Incidence')
plt.savefig("Graph_Incidence_vs_time_1500",dpi=300, bbox_inches='tight' )
plt.close()

In [2052]:
df66 = combinded6.median(axis = 1)
df66_2_5 = combinded6.quantile(q= 0.025, axis=1)
df66_25 = combinded6.quantile(q= 0.25, axis=1)
df66_75 = combinded6.quantile(q= 0.75, axis=1)
df66_97_5 = combinded6.quantile(q= 0.975, axis=1)

plt.plot(range(len(df66)),list(df66), label = 'Median')
plt.fill_between(range(len(df66)), np.array(df66)- (np.array(df66) - np.array(df66_25)), np.array(df66)+(np.array(df66_75) - np.array(df66)), alpha=0.2,edgecolor='#1B2ACC',
 facecolor='#089FFF', label =  r'$50\%$'" interval")
plt.fill_between(range(len(df66)), np.array(df66)- (np.array(df66) - np.array(df66_2_5)), np.array(df66)+(np.array(df66_97_5) - np.array(df66)), alpha =0.2, label =  r'$95\%$'" interval", edgecolor=(0,0,0,.4))
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.title(r'$\xi = 1500$')
plt.ylim(ymin = 0, ymax = 80)
plt.xlim(xmin = 0, xmax = 100)
plt.savefig("Daily incidence vs time one five",dpi=300, bbox_inches='tight')
plt.close()

In [2071]:
df11_mean = combinded6.mean(axis = 1)
df22_mean = combinded6.mean(axis = 1)
df33_mean = combinded6.mean(axis = 1)
df44_mean = combinded6.mean(axis = 1)
df55_mean = combinded6.mean(axis = 1)
df66_mean = combinded6.mean(axis = 1)

sample = pd.read_csv("SIRIncidence_1.csv")
plt.plot(range(len(df11)), df11, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df11_mean)), df11_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time1"], sample["Group1"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title('Control')
plt.savefig("Stochastic_vs_SIR_control",dpi=300, bbox_inches='tight')
plt.close()

plt.plot(range(len(df22)), df22, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df22_mean)), df22_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time2"], sample["Group2"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title(r'$\xi = 500$')
plt.savefig("Stochastic_vs_SIR_five",dpi=300, bbox_inches='tight')
plt.close()

plt.plot(range(len(df33)), df33, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df33_mean)), df33_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time3"], sample["Group3"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title(r'$\xi = 750$')
plt.savefig("Stochastic_vs_SIR_seven_five",dpi=300, bbox_inches='tight')
plt.close()

plt.plot(range(len(df44)), df44, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df44_mean)), df44_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time4"], sample["Group4"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title(r'$\xi = 1000$')
plt.savefig("Stochastic_vs_SIR_one_thou",dpi=300, bbox_inches='tight')
plt.close()

plt.plot(range(len(df55)), df55, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df55_mean)), df55_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time5"], sample["Group5"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title(r'$\xi = 1250$')
plt.savefig("Stochastic_vs_SIR_one_thou_two",dpi=300, bbox_inches='tight')
plt.close()

plt.plot(range(len(df66)), df66, color = "blue", label = 'Median stochastic SIR')
plt.plot(range(len(df66_mean)), df66_mean, color = "green", label = 'Mean stochastic SIR')
plt.plot(sample["Time6"], sample["Group6"], color = "black", label = 'Deterministic SIR')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.ylim(ymin = 0, ymax =140)
plt.title(r'$\xi = 1500$')
plt.savefig("Stochastic_vs_SIR_one_thou_five",dpi=300, bbox_inches='tight')
plt.close()


In [1885]:
plt.plot(sample["Time1"], sample["Group1"], color = "black", label = 'Group 1')
plt.plot(sample["Time2"], sample["Group2"], color = "purple", label = 'Group 2')
plt.plot(sample["Time3"], sample["Group3"], color = "red", label = 'Group 3')
plt.plot(sample["Time4"], sample["Group4"], color = "green", label = 'Group 4')
plt.plot(sample["Time5"], sample["Group5"], color = "blue", label = 'Group 5')
plt.plot(sample["Time6"], sample["Group6"], color = "grey", label = 'Group 6')
plt.ylabel("Daily Incidence")
plt.legend()
plt.xlabel("Time")
plt.xlim(xmin=0,xmax = 100)
plt.savefig("SIR_vs_Groups",dpi=300, bbox_inches='tight')
plt.close()

In [1911]:
max(df11),max(df22),max(df33),max(df44),max(df55),max(df66) #median

(43.0, 43.0, 43.0, 42.0, 42.0, 42.0)

In [1913]:
max(df11_mean),max(df22_mean),max(df33_mean),max(df44_mean),max(df55_mean),max(df66_mean) #average

(34.26589595375722,
 34.26589595375722,
 34.26589595375722,
 34.26589595375722,
 34.26589595375722,
 34.26589595375722)

In [1914]:
max(sample["Group1"]),max(sample["Group2"]),max(sample["Group3"]),max(sample["Group4"]),max(sample["Group5"]),max(sample["Group6"])

(100.934375434724,
 101.129983385135,
 102.62334322428,
 114.89339776964,
 113.423703738994,
 121.606340496726)

In [2104]:
print(sample[sample["Group1"] == max(sample["Group1"])].Time1.values)
print(sample[sample["Group2"] == max(sample["Group2"])].Time2.values)
print(sample[sample["Group3"] == max(sample["Group3"])].Time3.values)
print(sample[sample["Group4"] == max(sample["Group4"])].Time4.values)
print(sample[sample["Group5"] == max(sample["Group5"])].Time5.values)
print(sample[sample["Group6"] == max(sample["Group6"])].Time6.values)

[15.4]
[15.38]
[15.16]
[13.68]
[13.84]
[12.98]


In [2101]:
sample["Group1"]),max(sample["Group2"]),max(sample["Group3"]),max(sample["Group4"]),max(sample["Group5"]),max(sample["Group6"])



(365.0, 365.0, 365.0, 365.0, 365.0, 365.0)

Histograph of peak incidence and time of peak incidence for epidemics that took hold

In [2058]:
plt.hist(peak_incidence, color = "black", bins = 30)
#plt.title("Peak incidence per simulation")
plt.xlabel("Peak incidence")
plt.ylabel("Frequency")
plt.savefig("Histogram_peak_incidence_per_simulation",dpi=300, bbox_inches='tight')
plt.close()
plt.hist(time_peak_incidence, color = "black", bins = 30)
#plt.title("Time of peak incidence per simulation")
plt.xlabel("Time of peak incidence")
plt.ylabel("Frequency")
plt.savefig("Histogram_time_of_peak_incidence_per_simulation", dpi=300, bbox_inches='tight')
plt.close()

In [1931]:
max(time_peak_incidence), min(time_peak_incidence)

(65, 5)

Store peak incidence and time of peak incidence data per network 

In [1956]:
from itertools import accumulate
y = [len(x) for x in Epidemics_more_than_10] 
peak_incidence_pn = [peak_incidence[x - y: x] for x, y in zip(
        accumulate(y), y)]
time_peak_incidence_pn = [time_peak_incidence[x - y: x] for x, y in zip(
        accumulate(y), y)]

In [1957]:
peak_incidence_rolling_mean_3_pn = [peak_incidence_rolling_mean_3[x - y: x] for x, y in zip(
        accumulate(y), y)]
time_peak_incidence_rolling_mean_3_pn = [time_peak_incidence_rolling_mean_3[x - y: x] for x, y in zip(
        accumulate(y), y)]

In [1756]:
for i in range(net):
    p_i_pn = peak_incidence_pn[i]
    t_p_i = time_peak_incidence_pn[i]
    
    dt = {'Peak Incidence': p_i_pn, 'Time at peak incidence': t_p_i }
    dttt = pd.DataFrame(dt)
    dttt.to_csv('Incidence_parameters_pn_'+str(i)+'.csv', index = list(range(no_sim)))

Average incidence per network, average time for incidence per network

In [2079]:
Av_peak_incidence_per_network = []
Av_time_peak_incidence_per_network=[]
median_peak_incidence_per_network = []
median_time_peak_incidence_per_network=[]

for i in range(net):
    a_p_i_pn = np.sum(peak_incidence_pn[i])/len(peak_incidence_pn[i])
    a_t_p_i = np.sum(time_peak_incidence_pn[i])/len(time_peak_incidence_pn[i])
    m_p_i_pn = np.median(peak_incidence_pn[i])
    m_t_p_i = np.median(time_peak_incidence_pn[i])
    
    Av_peak_incidence_per_network.append(a_p_i_pn)
    Av_time_peak_incidence_per_network.append(a_t_p_i)
    
    median_peak_incidence_per_network.append(m_p_i_pn)
    median_time_peak_incidence_per_network.append(m_t_p_i)

In [1928]:
max(Av_peak_incidence_per_network), min(Av_peak_incidence_per_network)

(54.111111111111114, 45.49342105263158)

In [1965]:
median_peak_incidence_rolling_mean_3_per_network = []
av_peak_incidence_rolling_mean_3_per_network = []

Av_time_peak_incidence_rolling_mean_3_per_network=[]
median_time_peak_incidence_rolling_mean_3_per_network=[]

for i in range(net):
    a_p_i_pn = np.mean(peak_incidence_rolling_mean_3_pn[i])
    a_t_p_i = np.mean(time_peak_incidence_rolling_mean_3_pn[i])
    m_p_i_pn = np.median(peak_incidence_rolling_mean_3_pn[i])
    m_t_p_i = np.median(time_peak_incidence_rolling_mean_3_pn[i])
    
    av_peak_incidence_rolling_mean_3_per_network.append(a_p_i_pn)
    Av_time_peak_incidence_rolling_mean_3_per_network.append(a_t_p_i)
    
    median_peak_incidence_rolling_mean_3_per_network.append(m_p_i_pn)
    median_time_peak_incidence_rolling_mean_3_per_network.append(m_t_p_i)

Incidence results for networks with the same $\xi$ value and control networks

In [1758]:
peak_incidence_pg = list(np.array_split(np.array(peak_incidence), group)) 
time_peak_incidence_pg = list(np.array_split(np.array(time_peak_incidence), group))

In [1917]:
av_peak_incidence = [np.sum(i)/len(i) for i in peak_incidence_pg]
av_time_peak_incidence = [np.sum(i)/len(i) for i in time_peak_incidence_pg]

In [1988]:
median_peak_incidence = [np.median(i) for i in peak_incidence_pg]
median_time_peak_incidence = [np.median(i) for i in time_peak_incidence_pg]

In [1938]:
max_peak_inc = [max(i) for i in peak_incidence_pg]
min_peak_inc = [min(i) for i in peak_incidence_pg]
print(max_peak_inc)
print(min_peak_inc)

[64.75, 62.25, 62.25, 61.875, 59.25, 57.0]
[37.625, 35.75, 38.625, 34.375, 1.0, 40.0]


In [1934]:
max_time_peak_incidence =[max(i) for i in time_peak_incidence_pg]
max_time_peak_incidence

[48, 51, 55, 65, 52, 56]

In [1936]:
min_time_peak_incidence = [min(i) for i in time_peak_incidence_pg]
min_time_peak_incidence

[7, 6, 7, 6, 5, 6]

In [1930]:
np.round(av_time_peak_incidence,0)

array([18., 18., 18., 16., 15., 14.])

In [1922]:
np.round(av_time_peak_incidence,2)

array([17.76, 17.9 , 17.5 , 16.31, 15.46, 14.26])

In [1990]:
np.round(median_time_peak_incidence,2)

array([17. , 16. , 16. , 14.5, 13. , 12. ])

In [1989]:
np.round(median_peak_incidence,2)

array([52.38, 51.62, 51.25, 50.94, 50.25, 48.81])

Store results for network "groups"

In [1759]:
for i in range(group):
    p_i_pg = peak_incidence_pn[i]
    t_p_i_pg = time_peak_incidence_pn[i]
    data22 = {'Peak Incidence': p_i_pg, 'Time at peak incidence': t_p_i_pg }
    df22 = pd.DataFrame(data22)
    df22.to_csv('Incidence_parameters_pg_'+str(i)+'.csv', index = list(range(sim_p_group)))

Calculate variance, percentiles, and std for average peak incidence per network and store

In [1760]:
peak_incidence_variance = []
incidence_percentile_95 = []
incidence_percentile_5 =[]
incidence_std = []
for i in range(net):
    var = np.var(peak_incidence_pn[i])
    peak_incidence_variance.append(var)
    per95 = np.percentile(peak_incidence_pn[i],95)
    incidence_percentile_95.append(per95)
    per5 = np.percentile(peak_incidence_pn[i],5)
    incidence_percentile_5.append(per5)
    sd = np.std(peak_incidence_pn[i], ddof=1)/np.sqrt(len(peak_incidence_pn[i]))
    incidence_std.append(sd)

In [1991]:
peak_incidence_variance_pg = list(np.array_split(np.array(peak_incidence_variance), group)) 

In [1993]:
av_var_peak_incidence_pg = [np.mean(i) for i in peak_incidence_variance_pg]

In [1960]:
peak_incidence_rolling_mean_3_variance = []
incidence_rolling_mean_3_percentile_95 = []
incidence_rolling_mean_3_percentile_5 =[]
incidence_rolling_mean_3_std = []
for i in range(net):
    var = np.var(peak_incidence_rolling_mean_3_pn[i])
    peak_incidence_rolling_mean_3_variance.append(var)
    per95 = np.percentile(peak_incidence_rolling_mean_3_pn[i],95)
    incidence_rolling_mean_3_percentile_95.append(per95)
    per5 = np.percentile(peak_incidence_rolling_mean_3_pn[i],5)
    incidence_rolling_mean_3_percentile_5.append(per5)
    sd = np.std(peak_incidence_rolling_mean_3_pn[i], ddof=1)/np.sqrt(len(peak_incidence_rolling_mean_3_pn[i]))
    incidence_rolling_mean_3_std.append(sd)

In [1761]:
ind = list(range(1,net+1))
data9 = {'Average peak incidence per network': Av_peak_incidence_per_network,
        'Variance':  peak_incidence_variance,
        '5th percentile': incidence_percentile_5,
        '95th percentile': incidence_percentile_95,
        'Standard deviation': incidence_std}
df9 = pd.DataFrame(data9)
df9.to_csv('Average_peak_incidence_pn_vs_variance.csv', index = ind)

Peak incidence per network vs motifs, with percentiles

In [1762]:
yerr11=[np.array(Av_peak_incidence_per_network)- np.array(incidence_percentile_5), np.array(incidence_percentile_95)-np.array(Av_peak_incidence_per_network)]


In [1975]:
yerr11_33=[np.array(median_peak_incidence_per_network)- np.array(incidence_percentile_5), np.array(incidence_percentile_95)-np.array(median_peak_incidence_per_network)]


In [2133]:
plt.errorbar(Triangle_motif,  Av_peak_incidence_per_network, yerr=yerr11, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of triangle motifs")
plt.ylabel("Average peak incidence")
plt.ylim(ymin = 0)
#plt.title("The average peak incidence vs the number of Triangle motifs per network")
plt.savefig("Average_peak_incidence_vs_triangles", dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()

plt.errorbar(Diamond_motif,  Av_peak_incidence_per_network, yerr=yerr11, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of diamond motifs")
plt.ylabel("Average peak incidence")
#plt.title("The average peak incidence vs the number of Diamond motifs per network")
plt.ylim(ymin = 0)
plt.savefig("Average_peak_incidence_vs_Diamonds",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()

plt.errorbar(Tadpole_motif,  Av_peak_incidence_per_network, yerr=yerr11, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Average peak incidence")
plt.ylim(ymin = 0)
#plt.title("The average peak incidence vs the number of Tadpole motifs per network")
plt.savefig("Average_peak_incidence_vs_Tadpoles",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()


plt.errorbar(Bowtie_motif,  Av_peak_incidence_per_network, yerr=yerr11, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Average peak incidence")
plt.ylim(ymin = 0)
#plt.title("The average peak incidence vs the number of Bowtie motifs per network")
plt.savefig("Average_peak_incidence_vs_Bowtie",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()



In [2059]:
plt.errorbar(Triangle_motif, median_peak_incidence_per_network, yerr=yerr11_33, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of Triangle motifs")
plt.ylabel("Peak incidence")
plt.ylim(ymin = 0,ymax=70)
plt.xlim(xmin=0)
#plt.title("The average peak incidence vs the number of Triangle motifs per network")
plt.savefig("Median_peak_incidence_vs_triangles", dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()

plt.errorbar(Diamond_motif,  median_peak_incidence_per_network, yerr=yerr11_33, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of Diamond motifs")
plt.ylabel("Peak incidence")
#plt.title("The average peak incidence vs the number of Diamond motifs per network")
plt.ylim(ymin = 0,ymax=70)
plt.xlim(xmin=0)
plt.savefig("Median_peak_incidence_vs_Diamonds",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()

plt.errorbar(Tadpole_motif,  median_peak_incidence_per_network, yerr=yerr11_33, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of Tadpole motifs")
plt.ylabel("Peak incidence")
plt.ylim(ymin = 0,ymax=70)
plt.xlim(xmin=0)
#plt.title("The average peak incidence vs the number of Tadpole motifs per network")
plt.savefig("Median_peak_incidence_vs_Tadpoles",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()


plt.errorbar(Bowtie_motif,  median_peak_incidence_per_network, yerr=yerr11_33, markersize = 4, fmt = 'o',capsize=2, color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of Bowtie motifs")
plt.ylabel("Peak incidence")
plt.ylim(ymin = 0,ymax=70)
plt.xlim(xmin=0)
#plt.title("The average peak incidence vs the number of Bowtie motifs per network")
plt.savefig("Median_peak_incidence_vs_Bowtie",dpi=300, bbox_inches='tight')
#plt.yscale("log")
plt.close()

Variance, std and percentiles for time of peak incidence per network and store

In [1764]:
peak_incidence_time_variance = []
incidence_time_percentile_95 = []
incidence_time_percentile_5 =[]
incidence_time_std = []
for i in range(net):
    var = np.var(time_peak_incidence_pn[i])
    peak_incidence_time_variance.append(var)
    per95 = np.percentile(time_peak_incidence_pn[i],95)
    incidence_time_percentile_95.append(per95)
    per5 = np.percentile(time_peak_incidence_pn[i],5)
    incidence_time_percentile_5.append(per5)
    sd = np.std(time_peak_incidence_pn[i], ddof=1)/np.sqrt(len(time_peak_incidence_pn[i]))
    incidence_time_std.append(sd)

In [1998]:
peak_incidence_time_variance_pg = list(np.array_split(np.array(peak_incidence_time_variance), group)) 
av_peak_incidence_time_variance_pg = [np.mean(i) for i in peak_incidence_time_variance_pg]

In [1999]:
print(av_peak_incidence_time_variance_pg)
print(av_var_peak_incidence_pg )

[32.209655828361285, 44.06842845683125, 43.67122012721817, 53.17891824046581, 46.56541255037594, 49.809052853003735]
[20.582758471301386, 17.206250229133705, 17.27769388767252, 19.070316044642393, 10.52968343907131, 8.269466215871162]


In [1765]:
t = [min(x) for x in time_peak_incidence_pn]
p = [max(x) for x in time_peak_incidence_pn]

In [1766]:
#np.round(incidence_time_percentile_95,0)
#np.round(incidence_time_percentile_5,0)

In [1767]:
ind = list(range(1,net+1))
data91 = {'Average time at peak incidence per network': Av_time_peak_incidence_per_network,
        'Variance':  peak_incidence_time_variance,
        '5th percentile': incidence_time_percentile_5,
        '95th percentile': incidence_time_percentile_95,
        'Standard deviation': incidence_time_std}
df91 = pd.DataFrame(data91)
df91.to_csv('Average_peak_time_incidence_pn_vs_variance.csv', index = ind)

Average time of peak incidence per network, with percentiles plotted against motifs

In [2084]:
median_time_peak_incidence_per_network_pg = list(np.array_split(np.array(median_time_peak_incidence_per_network), group))
median_peak_incidence_per_network_pg = list(np.array_split(np.array(median_peak_incidence_per_network), group))

av_median_time_peak_incidence_per_network_pg = [np.mean(i) for i in median_time_peak_incidence_per_network_pg]
av_median_peak_incidence_per_network_pg = [np.mean(i) for i in median_peak_incidence_per_network_pg]

print(av_median_time_peak_incidence_per_network_pg)
print(av_median_peak_incidence_per_network_pg )


[16.85, 16.583333333333332, 15.75, 14.233333333333333, 13.2, 11.866666666666667]
[52.4375, 51.63125, 51.545833333333334, 50.979166666666664, 50.05416666666667, 48.59583333333333]


plot time of peak incidence per network vs triangles

In [1977]:
yerr_t11=[np.array(median_time_peak_incidence_per_network)- np.array(incidence_time_percentile_5), np.array(incidence_time_percentile_95)-np.array(median_time_peak_incidence_per_network)]


In [2134]:
plt.errorbar(Triangle_motif, Av_time_peak_incidence_per_network, yerr= yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of triangle motifs")
plt.ylabel("Average time of peak incidence")
#plt.title("Average time of peak incidence vs the number of Triangle motifs per network")
plt.ylim(ymin=0)
plt.savefig("Average_time_of_peak_incidence_vs_triangles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Diamond_motif,Av_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of diamond motifs")
plt.ylabel("Average time of peak incidence")
plt.ylim(ymin=0)
#plt.title("Average time of peak incidence vs the number of Diamond motifs per network")
plt.savefig("Average_time_of_peak_incidence_vs_Diamonds",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Tadpole_motif,Av_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Average time of peak incidence")
plt.ylim(ymin=0)
#plt.title("Average time of peak incidence vs the number of Tadpole motifs per network")
plt.savefig("Average_time_of_peak_incidence_vs_Tadpoles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Bowtie_motif,Av_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Average time of peak incidence")
plt.ylim(ymin=0)
#plt.title("Average time of peak incidence vs the number of Bowtie motifs per network")
plt.savefig("Average_time_of_peak_incidence_vs_Bowties",dpi=300, bbox_inches='tight')
plt.close()

In [2135]:

plt.errorbar(Triangle_motif, median_time_peak_incidence_per_network, yerr= yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of triangle motifs")
plt.ylabel("Time of peak incidence")
#plt.title("Average time of peak incidence vs the number of Triangle motifs per network")
plt.ylim(ymin=0)
plt.xlim(xmin=0)
plt.savefig("median_time_of_peak_incidence_vs_triangles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Diamond_motif,median_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of diamond motifs")
plt.ylabel("Time of peak incidence")
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Diamond motifs per network")
plt.savefig("median_time_of_peak_incidence_vs_Diamonds",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Tadpole_motif,median_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Time of peak incidence")
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Tadpole motifs per network")
plt.savefig("median_time_of_peak_incidence_vs_Tadpoles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Bowtie_motif,median_time_peak_incidence_per_network, yerr=yerr_t11, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Time of peak incidence")
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Bowtie motifs per network")
plt.savefig("median_time_of_peak_incidence_vs_Bowties",dpi=300, bbox_inches='tight')
plt.close()

### Time to 90% of total cases of infection

Note: index_no_ep is the simulation number for an epidemic that did not take hold

In [2142]:
time_to_90 = []
for j in range(no_tot_sim):
    
    if j not in index_no_ep: 
        sample = pd.read_csv(filenames2[j])
        infected = list(sample["Infected"])
        recovered = list(sample["Recovered"])
        susceptible = list(sample["Susceptible"])
        time = list(sample["Time"])
    
    
        percent_90_infected = round(recovered[-1]*0.9)
        y = susceptible[0] - percent_90_infected
        index = susceptible.index(y)
        time_to_90.append(time[index])

Calculating the number of epidemics that took hold for each network, making a list of times to $90\%$ of cases of infection for each network

In [2143]:
from itertools import accumulate
y = [len(x) for x in Epidemics_more_than_10] 
time_to_90_pn = [time_to_90[x - y: x] for x, y in zip(
        accumulate(y), y)]

In [2144]:
time_to_90_variance = []
time_to_90_percentile_95 = []
time_to_90_percentile_5 =[]
time_to_90_std = []

for i in range(net):
    var = np.var(time_to_90_pn[i])
    time_to_90_variance.append(var)
    
    per95 = np.percentile(time_to_90_pn[i],95)
    time_to_90_percentile_95.append(per95)
    
    per5 = np.percentile(time_to_90_pn[i],5)
    time_to_90_percentile_5.append(per5)
    
    sd = np.std(time_to_90_pn[i], ddof=1)/np.sqrt(len(time_to_90_pn[i]))
    time_to_90_std.append(sd)

In [2145]:
av_time_to_90 = []
for i in time_to_90_pn:
    p = np.sum(i)/len(i)
    av_time_to_90.append(p)

In [2147]:
median_time_to_90 = []
for i in time_to_90_pn:
    p = np.median(i)
    median_time_to_90.append(p)

In [2150]:
median_time_to_90_pg = list(np.array_split(np.array(mean_time_to_90), group)) 
var_av_time_to_90 = [np.var(i) for i in median_time_to_90_pg]

In [2151]:
var_av_time_to_90

[2.9758662302596135,
 3.2264921640502964,
 4.79973220915955,
 4.917301965228109,
 4.055655669390244,
 4.399758725695431]

In [1945]:
av_time_to_90_pg = list(np.array_split(np.array(av_time_to_90), group)) 
max_av_time_to_90 = [max(i) for i in av_time_to_90_pg]
min_av_time_to_90 = [min(i) for i in av_time_to_90_pg]

In [2099]:
time_to_90_variance_pg = list(np.array_split(np.array(time_to_90_variance), group)) 
av_time_to_90_variance_pg = [np.mean(i) for i in time_to_90_variance_pg ]

In [2100]:
av_time_to_90_variance_pg

[34.11129098551358,
 45.45890036379312,
 44.75694402417943,
 54.93981974861988,
 47.959277663977296,
 52.51414205184788]

In [1782]:
ind = list(range(1,net+1))
data91 = {'Average time to 90 cases of total infection pn':av_time_to_90,
        'Variance':  time_to_90_variance,
        '5th percentile': time_to_90_percentile_5 ,
        '95th percentile': time_to_90_percentile_95,
        'Standard deviation': time_to_90_std}
df91 = pd.DataFrame(data91)
df91.to_csv('Average_time_to_90_cases_infection_pn_vs_variance.csv', index = ind)

In [1939]:
plt.hist(time_to_90, align = "left", color = "black", bins = 28)
plt.xlabel("Average time to 90$\%$ of total cases of infection")
plt.ylabel("Frequency")
plt.savefig("histogram_av_time_to_90",dpi=300, bbox_inches='tight')
plt.close()

In [2148]:
yerr_tn1 =[np.array(median_time_to_90)- np.array(time_to_90_percentile_5), np.array(time_to_90_percentile_95)-np.array(median_time_to_90)]

plt.errorbar(Triangle_motif, median_time_to_90, yerr= yerr_tn1, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of triangle motifs")
plt.ylabel('Time to ' + r'$90\%$' + ' of total infections')
#plt.title("Average time of peak incidence vs the number of Triangle motifs per network")
plt.ylim(ymin=0)
plt.xlim(xmin=0)
plt.savefig("median_time_to_90_cases_vs_triangles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Diamond_motif,median_time_to_90, yerr=yerr_tn1, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of diamond motifs")
plt.ylabel('Time to ' + r'$90\%$' + ' of total infections')
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Diamond motifs per network")
plt.savefig("median_time_to_90_cases_vs_Diamonds",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Tadpole_motif,median_time_to_90, yerr=yerr_tn1, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of tadpole motifs")
plt.ylabel('Time to ' + r'$90\%$' + ' of total infections')
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Tadpole motifs per network")
plt.savefig("median_time_to_90_cases_vs_Tadpoles",dpi=300, bbox_inches='tight')
plt.close()

plt.errorbar(Bowtie_motif,median_time_to_90, yerr=yerr_tn1, markersize = 4, fmt = 'o',capsize=3,color='black',
             ecolor='gray', elinewidth=1)
plt.xlabel("Number of bowtie motifs")
plt.ylabel('Time to ' + r'$90\%$' + ' of total infections')
plt.ylim(ymin=0)
plt.xlim(xmin=0)
#plt.title("Average time of peak incidence vs the number of Bowtie motifs per network")
plt.savefig("median_time_to_90_cases_vs_Bowties",dpi=300, bbox_inches='tight')
plt.close()


## Proportion of epidemics that took hold per network "group"

In [1775]:
ep_took_hold = [len(x) for x in Epidemics_more_than_10]
ep_took_hold_pg = list(np.array_split(np.array(ep_took_hold), group)) # epidemics that took hold for network "groups" 
av_ep_took_hold_pg = [np.sum(x)/len(x) for x in y_pg] #average number of epidemics that took hold pg 
prop_ep_took_hold_pg = np.array(av_ep_took_hold_pg)/net_per_group

In [1419]:
shapiro(ep_took_hold_pg[0])[1] > 0.05, shapiro(ep_took_hold_pg[1])[1] > 0.05, shapiro(ep_took_hold_pg[2])[1] > 0.05,shapiro(ep_took_hold_pg[3])[1] > 0.05,shapiro(ep_took_hold_pg[4])[1] > 0.05,shapiro(ep_took_hold_pg[5])[1] > 0.05


(True, True, False, True, False, True)

In [963]:
t = [x/no_tot_sim for x in ep_took_hold_pg] #proportion of epidemics that took hold per network
std_pn_prop = [np.std(x) for x in ep_took_hold_pg] #std for the number of epidemics that took hold per network group

five_percentile_pn= [np.percentile(x,5) for x in t] #percentile for proportion of epidemics that took hold per "group"
ninty_five_percentile_pn = [np.percentile(x,95) for x in t]#percentile for proportion of epidemics that took hold per "group"

In [1089]:
yerr22 = [prop_ep_took_hold_pg  - np.array(five_percentile_pn), np.array(ninty_five_percentile_pn) - prop_ep_took_hold_pg]
plt.errorbar(["Control", r'$\xi = 500$' , r'$\xi = 750$',r'$\xi = 1000$',r'$\xi = 1250$',r'$\xi = 1500$'], prop_ep_took_hold_pg  , color = "black", yerr = np.array(yerr22)/30, markersize = 4, fmt = 'o',capsize=3,ecolor='gray', elinewidth=1)
plt.ylim(ymin=0)
plt.xlabel("Networks")
plt.ylabel("Average proportion of epidemics $>10$")
#plt.title("Average proportion of epidemics that took hold")
plt.savefig("av prop epidemics >10 with percentiles not at 0",dpi=300, bbox_inches='tight')
plt.close()

In [1533]:
yerr22 = [prop_ep_took_hold_pg  - five_percentile_pn, ninty_five_percentile_pn - prop_ep_took_hold_pg ]
plt.errorbar(["Control", r'$\xi = 500$' , r'$\xi = 750$',r'$\xi = 1000$',r'$\xi = 1250$',r'$\xi = 1500$'], prop_ep_took_hold_pg  , color = "black", yerr = np.array(yerr22)/30, markersize = 4, fmt = 'o',capsize=3,ecolor='gray', elinewidth=1)
#plt.ylim(ymin=0)
plt.xlabel("Networks")
plt.ylabel("Average proportion of epidemics")
#plt.title("Average proportion of epidemics that took hold")
plt.savefig("av prop epidemics >10 with percentiles",dpi=300, bbox_inches='tight')
plt.close()

## Proportion of epidemics that took hold in total

In [1788]:
Epidemics_took_hold = 0
for x in Epidemic_size:
    if x > ex_ep:
        Epidemics_took_hold +=1

print("Proportion of Epidemics that took hold =", Epidemics_took_hold/no_tot_sim)

Proportion of Epidemics that took hold = 0.667037037037037


### Proportion of epidemics took hold per network vs number of  motifs

In [975]:
prob_epidemics_took_hold_pn = np.array(ep_took_hold)/no_sim

In [2149]:

plt.scatter(Triangle_motif,prob_epidemics_took_hold_pn, color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylim(ymin=0, ymax=1)
plt.xlim(xmin=0)
plt.ylabel("Proportion of epidemics that take hold")
plt.title("Triangle motifs")
plt.savefig("Proportion_eps_took_hold_vs_triangles", dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Diamond_motif,prob_epidemics_took_hold_pn, color = "black")
plt.xlabel("Number of diamond motifs")
plt.ylim(ymin=0, ymax=1)
plt.xlim(xmin=0)
plt.ylabel("Proportion of epidemics that take hold")
plt.title("Diamond motifs")
plt.savefig("Proportion_eps_took_hold_vs_diamonds",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Tadpole_motif,prob_epidemics_took_hold_pn, color = "black")
plt.xlabel("Number of tadpole motifs")
plt.ylim(ymin=0, ymax=1)
plt.xlim(xmin=0)
plt.ylabel("Proportion of epidemics that take hold")
plt.title("Tadpole motifs")
plt.savefig("Proportion_eps_took_hold_vs_tadpoles",dpi=300, bbox_inches='tight')
plt.close()


plt.scatter(Bowtie_motif,prob_epidemics_took_hold_pn, color = "black")
plt.xlabel("Number of bowtie motifs")
plt.ylim(ymin=0, ymax=1)
plt.xlim(xmin=0)
plt.ylabel("Proportion of epidemics that take hold")
plt.title("Bowtie motifs")
plt.savefig("Proportion_eps_took_hold_vs_Bowties",dpi=300, bbox_inches='tight')
plt.close()

## Considering if we exploring the effect of isolates or motifs on epidemic outcomes 

In [66]:
filenames4 = []
for i in range(net):
    name = 'Network_'+str(i)+'subgraphs.csv'
    filenames4.append(name)

Maximum expected epidemic size per network

In [1811]:
max_ep_size_pn = []
for i in range(net):
    y = []
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        G = nx.from_numpy_matrix(adj_i)
        t = sorted(nx.connected_components(G), key=len, reverse=True)
        no_of_subgraphs = len(t)
        len_of_subgraphs = [len(x) for x in t ] 
        
        for j in range(no_of_subgraphs):
            prop_ep_per_subgroup = (len_of_subgraphs[j]**2)/n
            y.append(prop_ep_per_subgroup)
        
        max_ep_size = np.sum(y)
        max_ep_size_pn.append(max_ep_size)
        dat = {'Index':  list(range(no_of_subgraphs)),'Size of subgraphs': len_of_subgraphs}
        df0 = pd.DataFrame(dat)
        df0.to_csv('Network_'+ str(i)+ 'subgraphs.csv')

Maximum average epidemic size excluding subgraphs of size less than or equal to 10 

In [1812]:
#number_subgraphs_excl_10 = []
size_subgraphs_excl_10 = []
max_ep_size_excl_10_pn = []

for i in range(net):
    p = []
    #number_subgraphs_excl_10_pn = []
    size_subgraphs_excl_10_pn = []
    data = pd.read_csv(filenames4[i])
    no_sub = len(data["Size of subgraphs"])
    size_sub = data["Size of subgraphs"]
    
    for x in size_sub:
        if x > 10:
            size_subgraphs_excl_10_pn.append(x)
        
    size_subgraphs_excl_10.append(size_subgraphs_excl_10_pn) 
    #number_subgraphs_excl_10_pn.append(len(size_subgraphs_excl_10_pn)) 
    #number_subgraphs_excl_10.append(number_subgraphs_excl_10_pn)
    
    for k in range(len(size_subgraphs_excl_10_pn)):
        prop_epidemic_pn = (np.array(size_subgraphs_excl_10_pn[k]))**2/n
        p.append(prop_epidemic_pn)

    max_ep_siz = np.sum(p)
    max_ep_size_excl_10_pn.append(max_ep_siz)

In [1813]:
datt = {'Max epidemic size per network':  (max_ep_size_excl_10_pn),'Average epidemic size per network (excl 10)':av_ep_size_ex_10_pn}
df00 = pd.DataFrame(datt)
df00.to_csv('Max_ep_size_pn_vs_average_ep_size.csv')

In [2005]:
plt.scatter(Triangle_motif, np.array(av_ep_size_ex_10_pn)/np.array(max_ep_size_excl_10_pn), color = 'black')
plt.ylim(ymin=0, ymax = 1)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.title('Triangle motifs')
plt.xlabel('Number of triangle motifs')
plt.ylabel('Normalised epidemic size')
plt.savefig("max_mean_vs_average_ep_size_vs_triangles", dpi=400, bbox_inches='tight')
plt.close()

plt.scatter(Diamond_motif, np.array(av_ep_size_ex_10_pn)/np.array(max_ep_size_excl_10_pn), color = 'black')
plt.ylim(ymin=0, ymax = 1)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.title('Diamond motifs')
plt.xlabel('Number of diamond motifs')
plt.ylabel('Normalised epidemic size')
plt.savefig("max_mean_vs_average_ep_size_vs_diamond", dpi=400, bbox_inches='tight')
plt.close()


plt.scatter(Tadpole_motif, np.array(av_ep_size_ex_10_pn)/np.array(max_ep_size_excl_10_pn), color = 'black')
plt.ylim(ymin=0, ymax = 1)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.title('Tadpole motifs')
plt.xlabel('Number of tadpole motifs')
plt.ylabel('Normalised epidemic size')
plt.savefig("max_mean_vs_average_ep_size_vs_tadpoles", dpi=400, bbox_inches='tight')
plt.close()

plt.scatter(Bowtie_motif, np.array(av_ep_size_ex_10_pn)/np.array(max_ep_size_excl_10_pn), color = 'black')
plt.ylim(ymin=0, ymax = 1)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.title('Bowtie motifs')
plt.xlabel('Number of bowtie motifs')
plt.ylabel('Normalised epidemic size')
plt.savefig("max_mean_vs_average_ep_size_vs_bowties", dpi=400, bbox_inches='tight')
plt.close()

In [2006]:
plt.scatter(Triangle_motif, max_ep_size_excl_10_pn, color = 'black')
plt.ylim(ymin = 0,ymax = 1100)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Maximum average epidemic size")
#plt.title("Number of triangle motifs vs maximum average epidemic size")
plt.savefig("triangle_motis_vs_max_epidemic_size",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Bowtie_motif, max_ep_size_excl_10_pn, color = 'black')
plt.ylim(ymin = 0,ymax = 1100)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Maximum average epidemic size")
#plt.title("Number of triangle motifs vs maximum average epidemic size")
plt.savefig("bowtie_motis_vs_max_epidemic_size",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Diamond_motif, max_ep_size_excl_10_pn, color = 'black')
plt.ylim(ymin = 0, ymax = 1100)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.xlabel("Number of diamond motifs")
plt.ylabel("Maximum average epidemic size")
#plt.title("Number of triangle motifs vs maximum average epidemic size")
plt.savefig("diamond_motis_vs_max_epidemic_size",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Tadpole_motif, max_ep_size_excl_10_pn, color = 'black')
plt.ylim(ymin = 0,ymax = 1100)
plt.xlim(xmin = 0)
#plt.yscale("log")
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Maximum average epidemic size")
#plt.title("Number of triangle motifs vs maximum average epidemic size")
plt.savefig("tadpole_motis_vs_max_epidemic_size",dpi=300, bbox_inches='tight')
plt.close()

In [1816]:
av_ep_size_ex_10_pg = list(np.array_split(np.array(av_ep_size_ex_10_pn), group)) #average epidemic size per network split into per group

In [1817]:
max_ep_size_excl_10_pg = list(np.array_split(np.array(max_ep_size_excl_10_pn), group)) #maximum expected ep size pg

In [1818]:
av_pn = [np.sum(x)/len(x) for x in av_ep_size_ex_10_pg] # calculating average epidemic size per network type (network "group")
av_pn

[875.502992279502,
 861.1394496876832,
 837.2848990653608,
 805.8261412743029,
 762.8128681177398,
 704.3716819000772]

In [1819]:
av_std = [np.std(x,ddof = 1)  for x in av_ep_size_ex_10_pg]

In [1820]:
#av_std

In [1821]:
q75 = [np.percentile(x, 75) for x in av_ep_size_ex_10_pg]
q25 = [np.percentile(x, 25) for x in av_ep_size_ex_10_pg] 
np.array(q95) - np.array(q5)

array([14.86232974, 15.18218615, 15.03488095, 23.69753289, 27.36588698,
       29.03099275])

In [1822]:
slope, intercept, r, p, se = linregress(list(range(6)), av_pn)

In [986]:
min(av_ep_size_ex_10_pn), max(av_ep_size_ex_10_pn), min(max_ep_size_excl_10_pn), max(max_ep_size_excl_10_pn)

(677.1, 885.125, 824.464, 1000.0)

## Comparing properties of av path length, network diameter, clustering coefficient, degree distribtuion

In [108]:
av_clustering_coeff_pn = []
global_clustering_coeff_pn = []
for i in range(net):
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        G = nx.from_numpy_matrix(adj_i)
        
        average_shortest_path_l = []
        network_diameter = []
        p_k = []
        
        for C in (G.subgraph(c).copy() for c in nx.connected_components(G)):
            av_shortest_path_l = nx.average_shortest_path_length(C)
            net_diameter = nx.diameter(C)
            if av_shortest_path_l != 0:
                average_shortest_path_l.append(av_shortest_path_l)
            if  net_diameter != 0:
                network_diameter.append(net_diameter)
                
        s = adj_i.sum(axis = 1) 
        r = int(max(s))
        for j in range(r):
            prob = op.countOf(s,j)/n
            p_k.append(prob)
            
        da = {'Average shortest path length for subgraphs': average_shortest_path_l ,'Network Diameter for subgraphs': network_diameter}
        daa = {'P(k)': p_k, 'k': list(range(r))}
        dfa = pd.DataFrame(da)
        dfaa = pd.DataFrame(daa)
        dfa.to_csv('Network_'+str(i)+ '_path_length_properties.csv')
        dfaa.to_csv('Network_'+str(i)+ '_degree_distribution.csv')
        
        av_clustering_coeff = nx.average_clustering(G, nodes=None, weight=None, count_zeros=False)
        av_clustering_coeff_pn.append(av_clustering_coeff)
        
        global_clustering_coeff = 3*((1/6)*np.trace(adj_i**3))/((1/2)*np.sum([int(s[i])*(int(s[i])-1) for i in range(n)]))
        global_clustering_coeff_pn.append(global_clustering_coeff)

Store data for average clustering coefficients for networks

In [109]:
ind = list(range(net))
sample = {'Average clustering coefficient': av_clustering_coeff_pn, 'Global clustering coefficient':global_clustering_coeff_pn }
df = pd.DataFrame(sample)
df.to_csv('Clustering_coefficient_parameters_vs_network.csv', index = ind)

In [2016]:
sample = pd.read_csv('Clustering_coefficient_parameters_vs_network.csv')

plt.scatter(Triangle_motif, sample["Average clustering coefficient"], color = "black")
#plt.yscale("log")
plt.ylim(ymin = 0, ymax = 0.4)
plt.xlim(xmin = 0)
plt.xlabel("Number of triangle motifs")
plt.ylabel("Average clustering coefficient")

#plt.title("Average clustering coefficient excluding isolates per network")
plt.savefig("Average clustering coefficient excluding isolates per network",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Triangle_motif, sample["Global clustering coefficient"], color = "black")
#plt.yscale("log")
plt.ylim(ymin = 0, ymax = 0.13)
plt.xlim(xmin = 0)
plt.xlabel("Number of triangle motifs")
plt.ylabel("Global clustering coefficient")
#plt.title("Global clustering coefficient per network")
plt.savefig("Global clustering coefficient per network",dpi=300, bbox_inches='tight')
plt.close()

In [575]:
sample["Average clustering coefficient"][0], sample["Average clustering coefficient"][29]

(0.1204945987609015, 0.1605810438799894)

In [577]:
av_clust_coeff_pg = list(np.array_split(np.array(sample["Average clustering coefficient"]), group))

In [576]:
glob_clust_coeff_pg =  list(np.array_split(np.array(sample["Global clustering coefficient"]), group))

In [2087]:
mean_glob = [np.mean(x) for x in glob_clust_coeff_pg]
mean_glob

[0.01665532781358701,
 0.034428726546803796,
 0.05193785205947775,
 0.06469297227081448,
 0.08470635377052352,
 0.10126781156959058]

In [2086]:
mean_av = [np.mean(x) for x in av_clust_coeff_pg]
mean_av

[0.14768060931883617,
 0.16256970155765998,
 0.18837844168574946,
 0.22234849494795156,
 0.26175723940508494,
 0.29486683521085727]

## Exploring path length and network diameter properties

In [235]:
filenames7 = []
for i in range(net):
    name = 'Network_'+str(i)+ '_path_length_properties.csv'
    filenames7.append(name)

In [1907]:
max_path_subgraph_length = []
max_subgraph_diameter = []

for i in range(net):
    data = pd.read_csv(filenames7[i])
    av_short_path_l = data['Average shortest path length for subgraphs']
    network_diam = data['Network Diameter for subgraphs']
    max_av_short_path_l = max(av_short_path_l)
    max_path_subgraph_length.append(max_av_short_path_l)
    network_diam_max = max(network_diam)
    max_subgraph_diameter.append(network_diam_max)

In [1908]:
max_subgraph_diameter_pg = list(np.array_split(np.array(max_subgraph_diameter), group))

In [2019]:
plt.scatter(Triangle_motif, max_subgraph_diameter, color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Maximum sub-graph diameter")
plt.ylim(ymin = 0, ymax = 15)
plt.xlim(xmin = 0)
#plt.title("Maximum subgraph diameter vs number of triangle motifs per network")
plt.savefig("Maximum subgraph diameter per network vs number of triangle motifs per network",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Triangle_motif,  max_path_subgraph_length, color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylim(ymin = 0, ymax = 6)
plt.xlim(xmin = 0)
plt.ylabel("Maximum average sub-graph path length")
#plt.title("Maximum average sub-graph path length vs number of triangle motifs per network")
plt.savefig("Maximum average subgraph path length vs number of triangle motifs per network",dpi=300, bbox_inches='tight')
plt.close()

## Exploring degree distribution

In [237]:
filenames5 = []
for i in range(net):
    name = 'Network_'+str(i)+ '_degree_distribution.csv'
    filenames5.append(name)

Calculating the maximum degree for each network, to find the maximum degree for 180 networks

In [79]:
max_degree_k = []
for i in range(net):
    data = pd.read_csv(filenames5[i])
    degree_k = list(data["k"])
    max_degree = degree_k[-1]
    max_degree_k.append(max_degree)

In [1495]:
max_degree_k_pg = list(np.array_split(np.array(max_degree_k), group))

In [1528]:
av_deg_max_pg = [np.sum(x)/net_per_group for x in max_degree_k_pg] 
#av_deg_max_pg

Create a list for p(k) of each network

In [80]:
p_k =[]
for i in range(net):
    data = pd.read_csv(filenames5[i])
    p_kk = list(data["P(k)"])
    
    if len(p_kk) < (max(max_degree_k)+1):
        t = (max(max_degree_k)+1)-len(p_kk)
        for j in range(t):
            p_kk.append(0)
    
    p_k.append(p_kk)

In [81]:
p_k_pn = list(np.array_split(np.array(p_k), net))

Calculating average p(k) per group

In [82]:
p_k_pg = list(np.array_split(np.array(p_k), group))

In [85]:
av_p_k_g1 = np.zeros(max(max_degree_k)+1)
av_p_k_g2 = np.zeros(max(max_degree_k)+1)
av_p_k_g3 = np.zeros(max(max_degree_k)+1)
av_p_k_g4 = np.zeros(max(max_degree_k)+1)
av_p_k_g5 = np.zeros(max(max_degree_k)+1)
av_p_k_g6 = np.zeros(max(max_degree_k)+1)

for i in range(30):
    av_p_k_g1 = av_p_k_g1 +  p_k_pn[i]
for j in range(30,60):
    av_p_k_g2 = av_p_k_g2 +  p_k_pn[j]
for k in range(60,90):
    av_p_k_g3 = av_p_k_g3 +  p_k_pn[k]
for l in range(90,120):
    av_p_k_g4 = av_p_k_g4 +  p_k_pn[l]
for p in range(120,150):
    av_p_k_g5 = av_p_k_g5 +  p_k_pn[p]
for t in range(150,180):
    av_p_k_g6 = av_p_k_g6 +  p_k_pn[t]
    
average_pk_g1 = av_p_k_g1/net_per_group
average_pk_g2 = av_p_k_g2/net_per_group
average_pk_g3 = av_p_k_g3/net_per_group
average_pk_g4 = av_p_k_g4/net_per_group
average_pk_g5 = av_p_k_g5/net_per_group
average_pk_g6 = av_p_k_g6/net_per_group

Plotting degree distribution for control networks

In [2062]:
k = max(max_degree_k)+1
kk = list(range(k))
degree= list(range(1,k))
plt.scatter(kk,average_pk_g1, color = "black")
plt.plot(degree, [(1/(x**(2))) for x in degree], '--r', label = r'$\gamma = 2$')
plt.plot(degree, [(1/(x**(3))) for x in degree], '--b', label = r'$\gamma = 3$')
plt.xlabel("k")
plt.legend()
plt.ylabel("p(k)")
plt.xscale("log")
plt.yscale("log")
plt.title("Control group")
plt.savefig("degree_distribution_barabasi",dpi=300, bbox_inches='tight')
plt.close()

Ploting average for all rewired networks: 

In [88]:
av_pk_all_net = (av_p_k_g2 +av_p_k_g3 +av_p_k_g4 +av_p_k_g5 + av_p_k_g6)/(net-net_per_group)

In [2063]:
plt.scatter(kk,av_pk_all_net, color = "black")
plt.plot(degree, [(1/(x)**(2)) for x in degree ], '--r', label = r'$\gamma = 2$')
plt.plot(degree, [(1/(x)**(3)) for x in degree ], '--b', label = r'$\gamma = 3$')
plt.xlabel("k")
plt.legend()
plt.ylabel("p(k)")
plt.xscale("log")
plt.yscale("log")
plt.title("Rewired networks")
plt.savefig("degree_distribution_rewired_networks",dpi=300, bbox_inches='tight')
plt.close()

In [90]:
data = {"Average p(k) group 1":average_pk_g1[0],
       "Average p(k) group 2":average_pk_g2[0],
       "Average p(k) group 3":average_pk_g3[0],
       "Average p(k) group 4":average_pk_g4[0],
       "Average p(k) group 5":average_pk_g5[0],
       "Average p(k) group 6":average_pk_g6[0],
       "Average p(k) all networks": av_pk_all_net[0]}
df = pd.DataFrame(data)
df.to_csv('Average_degree_distribution_pg.csv', index = kk)

# Number of subgraphs per network and average per group

In [91]:
filenames4 = []
for i in range(net):
    name = 'Network_'+str(i)+'subgraphs.csv'
    filenames4.append(name)

In [1807]:
number_of_subgraphs_pn = []
for i in range(net):
    data = pd.read_csv(filenames4[i])
    no_sub = len(data["Number of Subgraphs"])
    number_of_subgraphs_pn.append(no_sub)

In [1808]:
number_of_subgraphs_pn.index(max(number_of_subgraphs_pn)), max(number_of_subgraphs_pn)

(156, 83)

In [2013]:
plt.scatter(Triangle_motif, number_of_subgraphs_pn, color = "black")
plt.title("Triangle motif")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Number of sub-graphs")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_subgraphs_vs_triangle",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Diamond_motif, number_of_subgraphs_pn, color = "black")
plt.title("Diamond motif")
plt.xlabel("Number of diamond motifs")
plt.ylabel("Number of sub-graphs")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_subgraphs_vs_diamond",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Tadpole_motif, number_of_subgraphs_pn, color = "black")
plt.title("Tadpole motif")
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Number of sub-graphs")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_subgraphs_vs_tadpole",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Bowtie_motif, number_of_subgraphs_pn, color = "black")
plt.title("Bowtie motif")
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Number of sub-graphs")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_subgraphs_vs_bowtie",dpi=300, bbox_inches='tight')
plt.close()

Average number of subgraphs per group

In [1803]:
av_subgraph_pg = []
sub_percentile_95_pg = []
sub_percentile_5_pg = []
subgraph_variance_pg =[]
subgraph_std_pg = []
number_of_subgraphs_pg = list(np.array_split(np.array(number_of_subgraphs_pn), group)) 
for i in range(group):
    av_sub = (np.sum(number_of_subgraphs_pg[i]))/net_per_group
    av_subgraph_pg.append(av_sub)
    std = np.std(number_of_subgraphs_pg[i])
    var = np.var(number_of_subgraphs_pg[i])
    per_95 = np.percentile(number_of_subgraphs_pg[i],95)
    per_5 = np.percentile(number_of_subgraphs_pg[i],5)
    subgraph_std_pg.append(std)
    subgraph_variance_pg.append(var)
    sub_percentile_95_pg.append(per_95)
    sub_percentile_5_pg.append(per_5)

In [2105]:
av_subgraph_pg

[1.0,
 1.4,
 4.733333333333333,
 13.233333333333333,
 32.266666666666666,
 68.76666666666667]

In [2118]:
size_of_subgraphs_pn_ = []
for i in range(net):
    size_of_subgraphs_pn = []
    data = pd.read_csv(filenames4[i])
    size_sub = np.array(data["Size of subgraphs"])
    for j in range(len(size_sub)):
        sub_size = size_sub[j]
        size_of_subgraphs_pn.append(sub_size)
    size_of_subgraphs_pn_.append(size_of_subgraphs_pn)

In [2125]:
max_size_of_subgraphs_pg = [max(i) for i in size_of_subgraphs_pn_]

In [2128]:
max_size_of_subgraphs_pg = list(np.array_split(np.array(max_size_of_subgraphs_pg), group)) 

In [2130]:
av_max_size_of_subgraphs_pg = [np.mean(i) for i in max_size_of_subgraphs_pg ]
av_max_size_of_subgraphs_pg

[1000.0,
 999.6,
 996.2333333333333,
 987.3666666666667,
 967.1666666666666,
 926.4666666666667]

In [1902]:
size_of_subgraphs_pg
size_of_subgraphs_pg_ex_1_pg =[]
for x in size_of_subgraphs_pg:
    size_of_subgraphs_pg_ex_1 =[]
    for j in x:
        if j>3:
            size_of_subgraphs_pg_ex_1.append(j)
    size_of_subgraphs_pg_ex_1_pg.append(size_of_subgraphs_pg_ex_1)

# Number of isolated nodes per network and average per group

In [1895]:
number_of_isolated_nodes_pn = []
for i in range(net):
    count = 0
    data = pd.read_csv(filenames4[i])
    l_sub = data["Size of subgraphs"]
    for i in range(len(l_sub)):
        if l_sub[i] == 1:
            count +=1
    number_of_isolated_nodes_pn.append(count)

In [1896]:
number_of_isolated_nodes_pn.index(max(number_of_isolated_nodes_pn)),max(number_of_isolated_nodes_pn)

(156, 77)

In [1897]:
number_of_isolated_nodes_pg = list(np.array_split(np.array(number_of_isolated_nodes_pn), group))
av_per_number_of_isolated_nodes_pg = [np.sum(x)/net_per_group for x in number_of_isolated_nodes_pg]
av_per_number_of_isolated_nodes_pg

[0.0, 0.4, 3.7, 11.866666666666667, 29.766666666666666, 62.8]

In [2014]:
plt.scatter(Triangle_motif, number_of_isolated_nodes_pn, color = "black")
plt.title("Triangle motif")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Number of isolated nodes")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_isolated_nodes_triangle",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Diamond_motif, number_of_isolated_nodes_pn, color = "black")
plt.title("Diamond motif")
plt.xlabel("Number of diamond motifs")
plt.ylabel("Number of isolated nodes")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_isolated_nodes_diamond",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Tadpole_motif, number_of_isolated_nodes_pn, color = "black")
plt.title("Tadpole motif")
plt.xlabel("Number of tadpole motifs")
plt.ylabel("Number of isolated nodes")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_isolated_nodes_tadpole",dpi=300, bbox_inches='tight')
plt.close()

plt.scatter(Bowtie_motif, number_of_isolated_nodes_pn, color = "black")
plt.title("Bowtie motif")
plt.xlabel("Number of bowtie motifs")
plt.ylabel("Number of isolated nodes")
plt.ylim(ymin = 0)
plt.xlim(xmin = 0)
plt.savefig("no_isolated_nodes_bowtie",dpi=300, bbox_inches='tight')
plt.close()

## Isolates and subgraphs compared with triangle motifs

In [1100]:
plt.scatter(Triangle_motif,number_of_isolated_nodes_pn, color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Number of isolated nodes")
plt.title("The number of triangle motifs vs number of isolated nodes per network")
plt.savefig("number_of_triangles_vs_isolated_nodes_pn",dpi=300, bbox_inches='tight')
plt.close()

In [1893]:
plt.scatter(Triangle_motif,number_of_subgraphs_pn, color = "black")
plt.xlabel("Number of triangle motifs")
plt.ylabel("Number of subgraphs")
plt.title("The number of triangle motifs vs number of subgraphs per network")
plt.savefig("no_triangles_vs_no_subgraphs_pn",dpi=300, bbox_inches='tight')
plt.close()

## SUB ANALYSIS

Probability of epidemics of size 1

In [110]:
num = 10**6
recover = expon.rvs(scale=gamma, size=num)
transmit = expon.rvs(scale=tau, size=num)
count = 0

for i in range(num):
    if recover[i]<transmit[i]:
        count +=1

epidemics_prob = count/num
print(epidemics_prob)

0.441311


### Proportion of epicemics with expected epidemic size 1:

### Using networks

In [1293]:
no_neighbour = []
one_neighbour = []
two_neighbour = []
three_neighbour = []
four_neighbour = []
five_neighbour = []
six_neighbour = []
seven_neighbour = []
eight_neighbour = []
nine_neighbour = []
ten_neighbour = []



for i in range(net):
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        G = nx.from_numpy_matrix(adj_i)
        
        s = adj_i.sum(axis = 1)
        
        no_neigh = op.countOf(s, 0)
        one_neigh = op.countOf(s, 1)
        two_neigh = op.countOf(s, 2)
        three_neigh = op.countOf(s, 3)
        four_neigh = op.countOf(s, 4)
        five_neigh = op.countOf(s, 5)
        six_neigh = op.countOf(s, 6)
        seven_neigh = op.countOf(s, 7)
        eight_neigh = op.countOf(s, 8)
        nine_neigh = op.countOf(s, 9)
        ten_neigh = op.countOf(s,10)
   
        one_neighbour.append(one_neigh)
        two_neighbour.append(two_neigh)
        three_neighbour.append(three_neigh)
        four_neighbour.append(four_neigh)
        five_neighbour.append(five_neigh)
        six_neighbour.append(six_neigh)
        seven_neighbour.append(seven_neigh)
        eight_neighbour.append(eight_neigh)
        nine_neighbour.append(nine_neigh)
        ten_neighbour.append(ten_neigh)

In [1536]:
prop_no = (np.sum(np.array(no_neighbour))/net)/n #net = 180 networks, n = 1000 nodes in each network
prop_one = (np.sum(np.array(one_neighbour))/net)/n
prop_two = (np.sum(np.array(two_neighbour))/net)/n
prop_three = (np.sum(np.array(three_neighbour))/net)/n
prop_four = (np.sum(np.array(four_neighbour))/net)/n
prop_five = (np.sum(np.array(five_neighbour))/net)/n
prop_six = (np.sum(np.array(six_neighbour))/net)/n
prop_seven = (np.sum(np.array(seven_neighbour))/net)/n
prop_eight = (np.sum(np.array(eight_neighbour))/net)/n
prop_nine = (np.sum(np.array(nine_neighbour))/net)/n
prop_ten = (np.sum(np.array(ten_neighbour))/net)/n

#m = tau/(tau+gamma) #p non-transmit 
m = gamma/(tau+gamma)

### Using degree distribution 

In [ ]:
p_k_0 = 0
p_k_1 = 0
p_k_2 = 0
p_k_3 = 0
p_k_4 = 0
p_k_5 = 0
p_k_6 = 0
p_k_7 = 0
p_k_8 = 0
p_k_9 = 0
p_k_10 = 0
for j in p_k_pn:
    for i in j:
        p_k_0 = p_k_0+i[0]
        p_k_1 = p_k_1+i[1]
        p_k_2 = p_k_2+i[2]
        p_k_3 = p_k_3+i[3]
        p_k_4 = p_k_4+i[4]
        p_k_5 = p_k_5+i[5]
        p_k_6 = p_k_6+i[6]
        p_k_7 = p_k_7+i[7]
        p_k_8 = p_k_8+i[8]
        p_k_9 = p_k_9+i[9]
        p_k_10 = p_k_10+i[10]

In [1843]:
p_k_0/net , p_k_1/net, p_k_2/net , p_k_3/net ,p_k_4/net ,p_k_5/net ,p_k_6/net ,p_k_7/net , p_k_8/net , p_k_9/net , p_k_10/net 

(0.018088888888888894,
 0.058727777777777775,
 0.11656666666666668,
 0.2285000000000001,
 0.15421111111111113,
 0.10143888888888895,
 0.07068333333333335,
 0.04946111111111108,
 0.03752222222222221,
 0.029399999999999996,
 0.022805555555555534)

In [2089]:
prop_ep_size_one = (0.01808 + prop_one*(m) + prop_two*(m**2) + prop_three*(m**3)+ prop_four*(m**4)+ prop_five*(m**5)+ prop_six*(m**6)+ prop_seven*(m**7)+ prop_eight*(m**8)+ prop_nine*(m**9) + prop_ten*(m**10))
prop_ep_size_one_2 = ((p_k_0/net) + (p_k_1/net)*(m) + (p_k_2/net)*(m**2) + (p_k_3/net)*(m**3)+ (p_k_4/net)*(m**4)+ (p_k_5/net)*(m**5)+ (p_k_6/net)*(m**6)+ (p_k_7/net)*(m**7)+ (p_k_8/net)*(m**8)+ (p_k_9/net)*(m**9) + (p_k_10/net)*(m**10))
print("Expected proportion of epidemics of size 1 = ", prop_ep_size_one)
print("Expected proportion of epidemics of size 1, using degree distribution = ", prop_ep_size_one_2)
print("Proportion of epidemics of size 1 = ", (op.countOf(Epidemic_size,1)/no_tot_sim))


Expected proportion of epidemics of size 1 =  0.1513183806916357
Expected proportion of epidemics of size 1, using degree distribution =  0.15132726958052461
Proportion of epidemics of size 1 =  0.26222222222222225


In [2090]:
prop_no,prop_one,prop_two,prop_three,prop_four,prop_five,prop_six,prop_seven,prop_eight,prop_nine,prop_ten 

(0.0,
 0.058727777777777775,
 0.11656666666666667,
 0.2285,
 0.1542111111111111,
 0.10143888888888888,
 0.07068333333333333,
 0.04946111111111111,
 0.03752222222222223,
 0.0294,
 0.022805555555555558)

Calculating expected R_0 for networks: 

In [1481]:
R_0_pn = []
for i in range(net):
    with open(filenames[i], 'r') as file:
        data_i = file.read()
        dat_i = np.matrix(data_i)
        adj_i = np.reshape(dat_i, (n, n))
        
        G = nx.from_numpy_matrix(adj_i)
        R_0 = EoN.estimate_R0(G, tau = 0.06, gamma = 0.076, transmissibility = 0.06/(0.06+0.076))
        R_0_pn.append(R_0)

In [1482]:
R_0_pg = list(np.array_split(np.array(R_0_pn), group))

In [1915]:
np.round(np.array(av_R_0_pg),2)

array([6.8 , 6.82, 6.88, 7.54, 7.46, 7.9 ])

In [1563]:
av_R_0_pg = [np.mean(x) for x in R_0_pg]
av_R_0_pg
np.sum(np.array(av_R_0_pg)*0.077)/6

0.5567409360148025

In [1562]:
np.sum(av_R_0_pg)/6

7.230401766426006